In [24]:
import pandas as pd
import numpy as np

In [30]:
pairwise_metrics = pd.read_csv('/Users/rushil/Downloads/pairwise_2x2_metrics_all_scans.csv')

# Create canonical pair by sorting the method names
pairwise_metrics[["method_min", "method_max"]] = (
    pairwise_metrics[["method_A", "method_B"]]
    .apply(lambda x: pd.Series(sorted(x)), axis=1)
)

# Drop duplicates using patient_id + stem + sorted pair
pairwise_metrics = pairwise_metrics.drop_duplicates(
    subset=["patient_id", "stem", "method_min", "method_max"]
)

# Now thresholding
threshold = 0.97
pairwise_metrics["above_thresh"] = (pairwise_metrics["dice"] > threshold).astype(int)

# ---- Method-level percentage ----
method_level = (
    pairwise_metrics.melt(
        id_vars=["patient_id", "above_thresh"],
        value_vars=["method_A", "method_B"],
        value_name="method"
    )
    .groupby("method")["above_thresh"]
    .mean()
    .mul(100)
    .reset_index()
    .rename(columns={"above_thresh": "pct_scans_above_0.97"})
)
# Round reported numbers to 1 decimal place
method_level = method_level.round(1)

# ---- Subject-level percentage ----
subject_level = (
    pairwise_metrics.melt(
        id_vars=["patient_id", "above_thresh"],
        value_vars=["method_A", "method_B"],
        value_name="method"
    )
    .groupby(["patient_id", "method"])["above_thresh"]
    .mean()
    .mul(100)
    .reset_index()
    .rename(columns={"above_thresh": "pct_scans_above_0.97"})
)
# Round reported numbers to 1 decimal place

subject_level_mean = (
    subject_level.groupby("method")["pct_scans_above_0.97"]
    .mean()
    .reset_index()
    .rename(columns={"pct_scans_above_0.97": "subject_avg_pct_scans_above_0.97"})
)

subject_level_mean = subject_level_mean.round(1)

print(method_level)
print(subject_level_mean)


         method  pct_scans_above_0.97
0     Brainchop                  19.2
1         CTBET                  47.4
2        CT_BET                  49.7
3  CTbet_Docker                  50.7
4      HD-CTBET                   3.3
5  Robust-CTBET                  50.1
6    SynthStrip                   5.1
         method  subject_avg_pct_scans_above_0.97
0     Brainchop                              19.5
1         CTBET                              47.5
2        CT_BET                              49.6
3  CTbet_Docker                              50.7
4      HD-CTBET                               3.4
5  Robust-CTBET                              50.0
6    SynthStrip                               5.1


In [32]:
df = pd.read_csv('/Users/rushil/Downloads/pairwise_2x2_metrics_all_scans.csv')

# 1) Deduplicate symmetric pairs (Dice/ICV are symmetric)
df[["method_min", "method_max"]] = (
    df[["method_A", "method_B"]]
    .apply(lambda x: pd.Series(sorted(x)), axis=1)
)
df = df.drop_duplicates(subset=["patient_id", "stem", "method_min", "method_max"])

# 2) Absolute ICV difference and indicator for ≤ 5 mL
df["abs_delta_icv_ml"] = df["delta_icv_ml"].abs()
df["within_5ml"] = (df["abs_delta_icv_ml"] <= 5).astype(int)

# 3) Method-level % of scans within 5 mL (count each method’s involvement)
method_icv_pct = (
    df.melt(
        id_vars=["patient_id", "stem", "within_5ml"],
        value_vars=["method_A", "method_B"],
        value_name="method"
    )
    .groupby("method")["within_5ml"]
    .mean()
    .mul(100)
    .reset_index()
    .rename(columns={"within_5ml": "pct_scans_abs_delta_icv_≤5ml"})
)

# 4) Subject-level % within 5 mL (patient × method)
subject_icv_pct = (
    df.melt(
        id_vars=["patient_id", "stem", "within_5ml"],
        value_vars=["method_A", "method_B"],
        value_name="method"
    )
    .groupby(["patient_id", "method"])["within_5ml"]
    .mean()
    .mul(100)
    .reset_index()
    .rename(columns={"within_5ml": "pct_scans_abs_delta_icv_≤5ml"})
)

# Optional: also report mean absolute ICV error (mL) by method and by subject
method_abs_mean = (
    df.melt(
        id_vars=["patient_id", "stem", "abs_delta_icv_ml"],
        value_vars=["method_A", "method_B"],
        value_name="method"
    )
    .groupby("method")["abs_delta_icv_ml"].mean().reset_index()
)

subject_abs_mean = (
    df.melt(
        id_vars=["patient_id", "stem", "abs_delta_icv_ml"],
        value_vars=["method_A", "method_B"],
        value_name="method"
    )
    .groupby(["patient_id", "method"])["abs_delta_icv_ml"].mean().reset_index()
)

subject_icv_pct_mean = (
    subject_icv_pct.groupby("method")["pct_scans_abs_delta_icv_≤5ml"]
    .mean()
    .reset_index()
    .rename(columns={"pct_scans_abs_delta_icv_≤5ml": "subject_avg_pct_scans_abs_delta_icv_≤5ml"})
)

method_icv_pct = method_icv_pct.round(1)
subject_icv_pct = subject_icv_pct.round(1)
method_abs_mean = method_abs_mean.round(1)
subject_abs_mean = subject_abs_mean.round(1)
subject_icv_pct_mean = subject_icv_pct_mean.round(1)

print(method_icv_pct)
print(subject_icv_pct_mean)


         method  pct_scans_abs_delta_icv_≤5ml
0     Brainchop                           6.2
1         CTBET                          21.6
2        CT_BET                          20.1
3  CTbet_Docker                          19.8
4      HD-CTBET                           4.0
5  Robust-CTBET                          13.8
6    SynthStrip                           1.4
         method  subject_avg_pct_scans_abs_delta_icv_≤5ml
0     Brainchop                                       6.4
1         CTBET                                      22.1
2        CT_BET                                      19.7
3  CTbet_Docker                                      20.3
4      HD-CTBET                                       3.9
5  Robust-CTBET                                      14.4
6    SynthStrip                                       1.4
